## The Battle of the Neighborhoods

This notebook will be used to solve the final assignment of the Coursera "Applied Data Science Capstone" course, __Battle of the Neighborhoods__. In this assignment I have to apply what I learn got through the "IBM Data Science Professional Certificate" and purpose a problem from my choice that I would like to solve using Foursquare data.

*Paulo Fernandes*

# A) Week 1 

### 1A) A description of the problem and a discussion of the background.

Moving to a different city is a decision that a lot of people have to deal with throughout their lives especially for young people looking for new and better jobs or going to college. Trying to make a decision about something new and unknown is always stressful but the best decisions usually have strong support with data. I'm gonna use this task to try to answer a problem that can be useful for these people including myself: __"If I get a job offer in a different city, what would be the best places to considerer moving in?"__  
Probably there will be more than one place that will fulfill our requirements, but in general, this place is gonna be as cheap as possible while complaining about our essential demands.  
The city that I will analyze will be Porto, Portugal. It is the second city in Portugal and a place that I'm considering moving in, so I will use this work to get more knowledge about the city and the renting business. The house pricing and rents in Portugal are increasing year over the year especially in Lisboa and Porto due to the concentration of opportunities in these areas. 

### 2A) A description of the data and how it will be used to solve the problem.

To solve this problem I build two datasets with the help of Google maps location [1] that will help me solving the problem:
- A dataset of 50 rooms and flats to rent in Porto with information I obtained from a web page of renting business [2];
- A the subway of Porto's subway stations because I didn't find any on the web and I considerer the distance from my place to a subway station an important measure; 

Both this datasets will be add to the GitHub repository.
To complete my work I'm gonna use the Foursquare data to get venues around the places that I select in order to try classify them. 
The goal is to try get the best deals from my personal perspective. Having a gym close by is a must be in my opinion, because after a day of work I love to go there and have my workout hour.  
As I have said already, the awser to this problem will be biased to my demandings. But that doesnt mean that the metodology that I'm gonna try to develop can not be used by other people with similar goals but diferent tasks. Maybe having a gym and a subway station close by isn't that important for you because you rather workout outside and having your own car. So you probably would considerer more important a place with garage or parking lot and close to a park.

#### Let's import the libraries.

In [39]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


#### Now, let's check and plot on a folium map both my built-in datasets locations:

In [40]:
address = 'Porto, Portugal'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ' + address +' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Porto, Portugal are 41.1494512, -8.6107884.


#### The subway stations locations:

In [41]:
#importing and clean the dataset:
dfSubway = pd.read_excel('SubwayPortoStations.xlsx')
dfSubway = dfSubway.dropna()
dfSubway.reset_index(drop=True, inplace=True)
dfSubway.head()

,Nome,Linha,Latitude,Longitude
0,Aeroporto,E,41.237455,-8.669518
1,Aliados,D,41.148587,-8.610948
2,Alto de Pêga,B,41.365287,-8.745053
3,Araújo,C,41.217939,-8.641021
4,Árvore,B,41.340756,-8.725598


In [42]:
#plot a follium map with the subway station of metro of Porto
map_porto_stations = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, name in zip(dfSubway['Latitude'], dfSubway['Longitude'], dfSubway['Nome']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_porto_stations)   
map_porto_stations

#### The renting places dataset:

In [43]:
dfRent = pd.read_excel('rentings.xlsx')
dfRent.reset_index(drop=True, inplace=True)
dfRent.head()

,Type,Furniture,Bills included,Latitude,Longitude,Price €/month
0,room,yes,yes,41.171966,-8.587178,410
1,room,yes,yes,41.191279,-8.615421,300
2,room,yes,no,41.166586,-8.597017,280
3,room,yes,yes,41.122323,-8.607831,350
4,room,yes,yes,41.152645,-8.621085,380


In [44]:
#plot a follium map of the places to rent in Porto
map_porto_rents = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, price in zip(dfRent['Latitude'], dfRent['Longitude'], dfRent['Price €/month']):
    label = '{}'.format(price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_porto_rents)  
map_porto_rents

#### To close my data section lets use the Foursquare API to get the venues going by 'gym/fitness' around a renting place:

In [45]:
#Foursquare credentials:
CLIENT_ID = 'MKSMC1YKOJXMX1Y44OUGCPE1LL4U0SFUYXTXYLFE0XFLNFYE' # your Foursquare ID
CLIENT_SECRET = 'KOYJSXPEBH0311KFDZV5X43WGOLQQPSHQ5M2AINCC0I0TFNN' # your Foursquare Secret
VERSION = '20202704'

In [46]:
#Lets get the location from the first renting place on the dataset:
latitude=dfRent['Latitude'][0]
longitude=dfRent['Longitude'][0]

#Now lets get the information using the Foursquare location and a radius of 1km
search_query = ['gym', 'fitness']
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

#Create a dataframe with the results
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.city,location.state
0,56e06bc2cd10ec50acbb9a1f,Elite Fitness studio,"[{'id': '52e81612bcbc57f1066b7a35', 'name': 'C...",v-1590393061,False,41.173997,-8.582970,"[{'label': 'display', 'lat': 41.1739973591177,...",418,PT,Portugal,[Portugal],NaN,NaN,NaN,NaN,NaN
1,50763fe7e4b065d50c3d2f57,FADEUP Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1590393061,False,41.172421,-8.598321,"[{'label': 'display', 'lat': 41.17242068503088...",935,PT,Portugal,"[Rua Dr. Plácido Costa, 91, Portugal]","Rua Dr. Plácido Costa, 91",NaN,NaN,NaN,NaN
2,5b75c7b0e4c459002c9fe450,Fitness Hut Antas,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1590393061,False,41.164012,-8.583066,"[{'label': 'display', 'lat': 41.164012, 'lng':...",950,PT,Portugal,[Rua dos Campeões Europeus de Viana (Rua Profe...,Rua dos Campeões Europeus de Viana,Rua Professor Manuel Baganha,4350-009,Porto,Porto


In [47]:
#Lets get the location from the second renting place on the dataset:
latitude=dfRent['Latitude'][1]
longitude=dfRent['Longitude'][1]

#Now lets get the information using the Foursquare location and a radius of 1km
search_query = ['gym', 'fitness']
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

#Create a dataframe with the results
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


""


As we can see, the first place on the dataset has 3 gym/fitness places within 1km while the seconds has none.

[1] https://www.google.com/maps  
[2] https://www.idealista.pt/